In [1]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import scipy
import sklearn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

d:\Anaconda\envs\mle_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\Anaconda\envs\mle_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\Anaconda\envs\mle_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\Anaconda\envs\mle_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [ ]:
# dataset_red=pd.read_csv("Data/红酒白酒分类+回归（都可以）/winequality-red.csv",sep=";",index_col=False)
# dataset_white=pd.read_csv("Data/红酒白酒分类+回归（都可以）/winequality-white.csv",sep=";",index_col=False)
dataset = pd.read_csv("Data/adult.data")
dataset.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'mmpy']
# dataset = pd.DataFrame(dataset, columns=['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'])
dataset.head()

In [ ]:
#preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(dataset["mmpy"])
X = dataset.drop("mmpy", axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=3)
pd.value_counts(y_test)

In [ ]:
#normalization
from sklearn.preprocessing import Normalizer
norm = Normalizer()
Xn_train = norm.fit_transform(X_train)
Xn_test = norm.transform(X_test)
Xn_test.shape

In [ ]:
#NN model
model = keras.models.Sequential()
model.add(keras.layers.Dense(256, input_dim=4, activation="linear"))
model.add(keras.layers.Dense(256, activation="linear"))
model.add(keras.layers.Dense(256, activation="linear"))
model.add(keras.layers.Dense(256, activation="linear"))
model.add(keras.layers.Dense(3, activation="softmax"))
model.summary()
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
#train the model
model.fit(np.array(Xn_train), np.array(y_train), epochs=1000, validation_data=(np.array(Xn_test), np.array(y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model.predict_classes(Xn_test)
print(y_pred)
print(y_test)
y_test = np.array(y_test)
y_test = y_test.reshape(y_test.shape[0], 1)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
scores = model.evaluate(np.array(Xn_test), np.array(y_test), verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [26]:
dataset_red=pd.read_csv("Data/红酒白酒分类+回归（都可以）/winequality-red.csv",sep=";",index_col=False)
dataset_white=pd.read_csv("Data/红酒白酒分类+回归（都可以）/winequality-white.csv",sep=";",index_col=False)
# dataset_red.head(n=len(dataset_red))
red=["red"]*len(dataset_red)
dataset_red.insert(0,"Label",red)
white=["white"]*len(dataset_white)
dataset_white.insert(0,"Label",white)
# dataset_white = dataset_white[:1599]
dataset=pd.concat([dataset_red,dataset_white],axis=0)
dataset.set_index("Label")
dataset["Label"].value_counts()

white    4898
red      1599
Name: Label, dtype: int64

In [27]:
#preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset["Label"]=le.fit_transform(dataset["Label"])
y = dataset["Label"]
X = dataset.drop('Label',axis=1)

#split the dataset into trainning set and testing set
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)
# dataset['Label'].value_counts(normalize=True)
X
print(y.value_counts())
print(y_train.value_counts())
print(y_test.value_counts())

1    4898
0    1599
Name: Label, dtype: int64
1    3930
0    1267
Name: Label, dtype: int64
1    968
0    332
Name: Label, dtype: int64


In [28]:
#Normalization
from sklearn.preprocessing import StandardScaler
norm = StandardScaler()
norm = norm.fit(X_train)
Xn_train = norm.transform(X_train)
Xn_test = norm.transform(X_test)
Xn_train.shape

(5197, 12)

In [ ]:
import numpy as np
corrMatt = dataset.corr()
# print(corrMatt)  相关矩阵

mask = np.zeros_like(corrMatt)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(20, 12))
plt.title('Wine feature Correlation')

cmap = sns.diverging_palette(260, 10, as_cmap=True)

sns.heatmap(corrMatt, vmax=1.2, square=False, cmap=cmap, mask=mask,
ax=ax, annot=True, fmt='.2g', linewidths=1);

In [29]:
#Feature selection
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif,k=4)
Xnf_train = selector.fit_transform(Xn_train,y_train)
Xnf_test = selector.transform(Xn_test)
Xnf_train

array([[ 1.53480958, -0.36693553, -0.40019175,  0.36752272],
       [ 1.53480958,  0.53943052,  0.87634441, -0.00462901],
       [-0.62228741,  1.38537216,  1.86275871, -1.42234986],
       ...,
       [-1.23860083, -0.97117956, -0.51624049, -0.57171735],
       [-0.23709152, -0.24608672, -0.48722831,  0.10170006],
       [-0.46820905, -1.27330158,  0.06400321,  0.26119365]])

In [30]:
#defining ANN model with two hidden layers, first hidden layer has 20 neurons, the next one has 10 neurons
model=keras.models.Sequential()
model.add(keras.layers.Dense(256, input_dim=4,activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
# model.add(keras.layers.Dense(512, activation="linear"))
model.add(keras.layers.Dense(1,activation='sigmoid'))
# model.add(keras.layers.Dense(1, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 256)               1280      
_________________________________________________________________
dense_21 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_22 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_23 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 257       
Total params: 198,913
Trainable params: 198,913
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
clf = keras.models.Sequential()
clf.add(keras.layers.Dense(50, input_dim=8, activation='relu'))
clf.add(keras.layers.Dense(100, activation='relu'))
clf.add(keras.layers.Dense(100, activation='relu'))
clf.add(keras.layers.Dense(1, activation='sigmoid'))
clf.summary()

In [ ]:
clf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
#train the model
history = model.fit((np.array(Xnf_train), np.array(y_train)), epochs=100, validation_data=(np.array(Xnf_test), np.array(y_test)))

Train on 5197 samples, validate on 1300 samples
Epoch 1/100
5197/5197 [==============================] - 1s 162us/sample - loss: 0.0178 - acc: 1.0000 - val_loss: 12.2614 - val_acc: 0.7446
Epoch 2/100
5197/5197 [==============================] - 1s 105us/sample - loss: 3.0572e-08 - acc: 1.0000 - val_loss: 12.2659 - val_acc: 0.7446
Epoch 3/100
5197/5197 [==============================] - 1s 100us/sample - loss: 3.0364e-08 - acc: 1.0000 - val_loss: 12.2719 - val_acc: 0.7446
Epoch 4/100
5197/5197 [==============================] - 0s 93us/sample - loss: 3.0119e-08 - acc: 1.0000 - val_loss: 12.2791 - val_acc: 0.7446
Epoch 5/100
5197/5197 [==============================] - 1s 100us/sample - loss: 2.9814e-08 - acc: 1.0000 - val_loss: 12.2870 - val_acc: 0.7446
Epoch 6/100
5197/5197 [==============================] - 1s 97us/sample - loss: 2.9464e-08 - acc: 1.0000 - val_loss: 12.2966 - val_acc: 0.7446
Epoch 7/100
5197/5197 [==============================] - 0s 96us/sample - loss: 2.9077e-08 - a

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = model.predict_classes(Xnf_test)
y_test = np.array(y_test)
print(y_pred)
print(y_test)
y_test = y_test.reshape((np.array(y_test)).shape[0], 1)
cm = confusion_matrix(np.array(y_test), y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
[0 0 1 ... 0 1 1]
[[  0 332]
 [  0 968]]


0.7446153846153846

In [ ]:
scores = model.evaluate(np.array(Xn_test), np.array(y_test), verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))